Required Packages: 
- pandas
- numpy
- matplotlib (pyplot from it)
- scikiet-learn
- wbgapi (world bank data api)

Main
conda install -y pandas numpy matplotlib scikit-learn jupyterlab

World bank API client
pip install wbgapi

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import wbgapi as wb

print(os.getcwd())

/Users/mnn/Documents/ME/Professional Porfolios/Data Analytics Portfolio/AIxSustainability


Data Sources:
- CO2 Emission per capita: Sourced from Our World in Data, using OWID offical github repository. 
- Renewable Energy Consumption (% of total energy consumption) - EG.FEC.RNEW.ZS: Sourced from World Bank using API client wbgapi 
- Life Expectancy at Birth – SP.DYN.LE00.IN: : Sourced from World Bank using API client wbgapi 
- Mean Years of schooling: Sourced from UNDP HDI data webpage through csv download
- Gross National Income per capita: Sourced from UNDP HDI data webpage through csv download
- Access to electricity – EG.ELC.ACCS.ZS: Sourced from World Bank using API client wbgapi
- Urban population (% of population) – SP.URB.TOTL.IN.ZS: Sourced from World Bank using API client wbgapi 
- Total population growth rate - SP.POP.GROW: Sourced from World Bank using API client wbgapi 
- Mortality Rate under-5 years of age population – SH.DYN.MORT: Sourced from World Bank using API client wbgapi

In [2]:
#UNDP DATA for Mean Years of Schooling and Gross National Income per Capita (2021PPP$)

mys_path = "mys.xlsx"
gni_path = "GNI_pc.xlsx"

mys = pd.read_excel("mys.xlsx")
gni = pd.read_excel("GNI_pc.xlsx")

display(mys.head())
display(gni.head())

,countryIsoCode,country,indexCode,index,dimension,indicatorCode,indicator,year,yearStr,actualValue,value,note
0,AFG,Afghanistan,HDI,Human Development Index,NaN,mys,Mean Years of Schooling (years),1990,1990,0.871962,0.872,NaN
1,AFG,Afghanistan,HDI,Human Development Index,NaN,mys,Mean Years of Schooling (years),1991,1991,0.915267,0.915,NaN
2,AFG,Afghanistan,HDI,Human Development Index,NaN,mys,Mean Years of Schooling (years),1992,1992,0.958573,0.959,NaN
3,AFG,Afghanistan,HDI,Human Development Index,NaN,mys,Mean Years of Schooling (years),1993,1993,1.001878,1.002,NaN
4,AFG,Afghanistan,HDI,Human Development Index,NaN,mys,Mean Years of Schooling (years),1994,1994,1.045184,1.045,NaN


,countryIsoCode,country,indexCode,index,dimension,indicatorCode,indicator,year,yearStr,actualValue,value,note
0,AFG,Afghanistan,HDI,Human Development Index,NaN,gnipc,Gross National Income Per Capita (2021 PPP$),1990,1990,3642.049616,3642.050,NaN
1,AFG,Afghanistan,HDI,Human Development Index,NaN,gnipc,Gross National Income Per Capita (2021 PPP$),1991,1991,3192.813684,3192.814,NaN
2,AFG,Afghanistan,HDI,Human Development Index,NaN,gnipc,Gross National Income Per Capita (2021 PPP$),1992,1992,2990.156395,2990.156,NaN
3,AFG,Afghanistan,HDI,Human Development Index,NaN,gnipc,Gross National Income Per Capita (2021 PPP$),1993,1993,1993.698011,1993.698,NaN
4,AFG,Afghanistan,HDI,Human Development Index,NaN,gnipc,Gross National Income Per Capita (2021 PPP$),1994,1994,1425.203987,1425.204,NaN


In [3]:
# Long to Wide format for undp datasets and merging them into one

mys = mys[["countryIsoCode", "country", "year", "actualValue"]] # keep only relevant columns

# Naming conventions for iso code column and renaming value column to the indicator name
mys = mys.rename(
    columns={
        "countryIsoCode": "iso_code",
        "actualValue": "mys"
    }
)

gni = (
    gni[["countryIsoCode", "country", "year", "actualValue"]]
    .rename(columns={"countryIsoCode": "iso_code",
                     "actualValue": "gni_pc"})
)

undp_df = mys.merge(
    gni,
    on=["iso_code", "year"],                # Match using key identifier of (iso code and year). Since both columns have iso_code and year column
    how="outer",                             # Row identities = union of left and right keys → matching keys → one row, both sides fill. Left-only keys → left fills, right = NaN. Right-only keys → right fills, left = NaN
    suffixes=("_mys", "_gni")               # For cols name that exist in both such as country
)

# Use country from MYS where available, otherwise fall back to GNI's country
undp_df["country"] = undp_df["country_mys"].fillna(undp_df["country_gni"])

# Drop the temporary country columns
undp_df = undp_df.drop(columns=["country_mys", "country_gni"])

display(undp_df.head())

,iso_code,year,mys,gni_pc,country
0,AFG,1990,0.871962,3642.049616,Afghanistan
1,AFG,1991,0.915267,3192.813684,Afghanistan
2,AFG,1992,0.958573,2990.156395,Afghanistan
3,AFG,1993,1.001878,1993.698011,Afghanistan
4,AFG,1994,1.045184,1425.203987,Afghanistan


In [4]:
# CO2 Emissions Data from Our World in Data, using github repository

# Loading dataset information before to only select the neccecary variable instead of requesting entire dataset, reducing time.
codebook_path = "https://raw.githubusercontent.com/owid/co2-data/master/owid-co2-codebook.csv"

codebook = pd.read_csv(codebook_path)

display(codebook.head())

codebook[codebook["column"].str.contains("co2", case=False, na=False)][
    ["column","title","description","unit"]
].head(20)

,column,title,description,unit,source
0,country,Country,Geographic location.,NaN,Our World in Data - Regions (2024)
1,year,Year,Year of observation.,NaN,Our World in Data - Regions (2024)
2,iso_code,ISO code,ISO 3166-1 alpha-3 three-letter country codes.,NaN,International Organization for Standardization...
3,population,Population,"Population by country, available from 10,000 B...",people,Population based on various sources (2024) [ht...
4,gdp,Gross domestic product (GDP),Total economic output of a country or region p...,international-$ in 2011 prices ($),Bolt and van Zanden – Maddison Project Databas...


,column,title,description,unit
5,cement_co2,Annual CO₂ emissions from cement,Annual emissions of carbon dioxide (CO₂) from ...,million tonnes (Mt)
6,cement_co2_per_capita,Annual CO₂ emissions from cement (per capita),Annual emissions of carbon dioxide (CO₂) from ...,tonnes per person (t/person)
7,co2,Annual CO₂ emissions,Annual total emissions of carbon dioxide (CO₂)...,million tonnes (Mt)
8,co2_growth_abs,Annual CO₂ emissions growth (abs),Annual growth in total emissions of carbon dio...,million tonnes (Mt)
9,co2_growth_prct,Annual CO₂ emissions growth (%),Annual percentage growth in total emissions of...,%
10,co2_including_luc,Annual CO₂ emissions including land-use change,Annual total emissions of carbon dioxide (CO₂)...,million tonnes (Mt)
11,co2_including_luc_growth_abs,Growth rate of emissions including land-use ch...,Annual growth in total emissions of carbon dio...,million tonnes (Mt)
12,co2_including_luc_growth_prct,Growth rate of emissions including land-use ch...,Annual percentage growth in total emissions of...,%
13,co2_including_luc_per_capita,Annual CO₂ emissions including land-use change...,"Annual emissions of carbon dioxide (CO₂), incl...",tonnes per person (t/person)
14,co2_including_luc_per_gdp,Annual CO₂ emissions including land-use change...,Annual total emissions of carbon dioxide (CO₂)...,kilograms per international-$ (kg/$)


In [5]:
# Relevant variables
cols = [
    "country",
    "iso_code",
    "year",
    "co2",
    "co2_per_capita",
    "co2_growth_prct",
    "consumption_co2",
]

co2_path = "https://raw.githubusercontent.com/owid/co2-data/master/owid-co2-data.csv"

co2 = pd.read_csv(co2_path, usecols=cols)

# Filter for year from 1990 to latest
co2 = co2[co2["year"] >= 1990] 


# Remove aggregates / regions from OWID 
# Drop rows with missing iso_code (these are things like "Africa", "Asia", "World")
co2 = co2[co2["iso_code"].notna()]
# Drop rows where iso_code starts with 'OWID_' (OWID aggregate codes)
co2 = co2[~co2["iso_code"].str.startswith("OWID_")]


display(co2.head())

,country,year,iso_code,co2,co2_growth_prct,co2_per_capita,consumption_co2
240,Afghanistan,1990,AFG,2.024,-26.784,0.168,NaN
241,Afghanistan,1991,AFG,1.914,-5.435,0.156,NaN
242,Afghanistan,1992,AFG,1.482,-22.580,0.112,NaN
243,Afghanistan,1993,AFG,1.487,0.330,0.100,NaN
244,Afghanistan,1994,AFG,1.454,-2.227,0.089,NaN


In [6]:
# The indicators required from World Bank API

#A Dict of indicator codes as dict key and remapping names as value of dict. 
wb_indicators = {
    "EG.FEC.RNEW.ZS": "renewable_energy_pct",
    "SP.DYN.LE00.IN": "life_expectancy",
    "EG.ELC.ACCS.ZS": "electricity_access_pct",
    "SP.URB.TOTL.IN.ZS": "urban_population_pct",
    "SP.POP.GROW": "population_growth_pct",
    "SH.DYN.MORT": "under5_mortality",
}

#Requesting data for choosen indicators from years 1990 till

wb_df = wb.data.DataFrame(
    series=list(wb_indicators.keys()),  # which indicators
    time=range(1990, 2026),             # from 1990,...,2025)
    labels=False,                       # keep ISO-3 codes instead of names
    skipBlanks=False,                   # keep rows even if values are missing
    columns='series',                   # one column per indicator code
    numericTimeKeys=True,               # Gives int years instead of str years like "YR2020"
    skipAggs=True                       # drop aggregates, keep only countries
)

# From MultiIndex (economy, time) to normal columns
wb_df = wb_df.reset_index()

# Rename economy/time to iso_code/year
wb_df = wb_df.rename(columns={
    "economy": "iso_code",
    "time": "year"
})

# Rename indicator columns from codes to defined names
wb_df = wb_df.rename(columns=wb_indicators)

# Adding country names
meta = wb.economy.DataFrame()[["name"]] #gets metadata for economies, however, only first column that is the name of country
meta = meta.reset_index() #iso came as index, so flattening it for later use

meta = meta.rename(columns={
    meta.columns[0]: "iso_code",  # .columns will help return metadate of column names for dataframe
    "name": "country"             # name to country
})


# left identities (iso_code from wb_df) define rows → all wb_df rows remain → meta columns attach only when iso_code matches → no missing meta values because every wb_df iso_code exists in meta

wb_df = wb_df.merge(
    meta,
    on='iso_code',  # column on both sides now. Otherwise would need to use left_on='iso_code': column on the left (wb_df) and right_index=True: index on the right (index of meta)
    how='left'      # left identities (iso_code from wb_df) define rows → all wb_df rows remain → meta columns attach only when iso_code matches → no missing meta values because every wb_df iso_code exists in meta
)

display(wb_df.head())

,iso_code,year,electricity_access_pct,renewable_energy_pct,under5_mortality,life_expectancy,population_growth_pct,urban_population_pct,country
0,ABW,1990,100.000000,0.3,NaN,72.546,3.750561,65.432816,Aruba
1,ABW,1991,99.153656,0.2,NaN,72.592,4.887136,65.400922,Aruba
2,ABW,1992,99.197128,0.2,NaN,72.717,4.610122,65.390589,Aruba
3,ABW,1993,99.239914,0.2,NaN,72.777,6.562028,65.384467,Aruba
4,ABW,1994,100.000000,0.2,NaN,72.796,5.170374,65.380048,Aruba


In [7]:
# Making a full raw dataset

# Merging co2 + wb_df
raw_data = co2.merge(
    wb_df,
    on=["iso_code", "year"],  # match by country code + year
    how="outer",              # keep all rows even if some values are missing
    suffixes=("", "_wb")      # if there are duplicate column names, add "_wb" to wb_df ones
)

# Then merge in undp_df (with mys, gni_pc, etc.)
raw_data = raw_data.merge(
    undp_df,
    on=["iso_code", "year"],  # same keys
    how="outer",              # again keep everything
    suffixes=("", "_undp")    # duplicate cols from undp_df get "_undp"
)

display(raw_data.head())

,country,year,iso_code,co2,co2_growth_prct,co2_per_capita,consumption_co2,electricity_access_pct,renewable_energy_pct,under5_mortality,life_expectancy,population_growth_pct,urban_population_pct,country_wb,mys,gni_pc,country_undp
0,Aruba,1990,ABW,0.487,-24.859,7.561,NaN,100.000000,0.3,NaN,72.546,3.750561,65.432816,Aruba,NaN,NaN,NaN
1,Aruba,1991,ABW,0.531,9.023,7.981,NaN,99.153656,0.2,NaN,72.592,4.887136,65.400922,Aruba,NaN,NaN,NaN
2,Aruba,1992,ABW,0.539,1.379,7.818,NaN,99.197128,0.2,NaN,72.717,4.610122,65.390589,Aruba,NaN,NaN,NaN
3,Aruba,1993,ABW,0.649,20.408,9.123,NaN,99.239914,0.2,NaN,72.777,6.562028,65.384467,Aruba,NaN,NaN,NaN
4,Aruba,1994,ABW,0.660,1.695,8.976,NaN,100.000000,0.2,NaN,72.796,5.170374,65.380048,Aruba,NaN,NaN,NaN


In [8]:
def summarize_ids(df, name, iso_col="iso_code", country_col="country"):
    rows, cols = df.shape
    n_iso = df[iso_col].nunique(dropna=True)
    n_ctry = df[country_col].nunique(dropna=True)
    miss_iso = df[iso_col].isna().sum()
    miss_ctry = df[country_col].isna().sum()
    
    print(
        f"{name}: shape={rows}x{cols}, "
        f"uniq {iso_col}={n_iso}, uniq {country_col}={n_ctry}, "
        f"missing {iso_col}={miss_iso}, missing {country_col}={miss_ctry}"
    )


summarize_ids(mys,      "UNDP MYS",    iso_col="iso_code", country_col="country")
summarize_ids(gni,      "UNDP GNI",    iso_col="iso_code", country_col="country")
summarize_ids(undp_df,  "UNDP merged", iso_col="iso_code", country_col="country")
summarize_ids(co2,      "OWID CO2",    iso_col="iso_code", country_col="country")
summarize_ids(wb_df,    "World Bank indicators", iso_col="iso_code", country_col="country")
summarize_ids(raw_data, "raw_data merged (co2 + wb_df + undp_df)", iso_col="iso_code", country_col="country")

UNDP MYS: shape=6455x4, uniq iso_code=204, uniq country=204, missing iso_code=0, missing country=0
UNDP GNI: shape=6852x4, uniq iso_code=204, uniq country=204, missing iso_code=0, missing country=0
UNDP merged: shape=6893x5, uniq iso_code=204, uniq country=204, missing iso_code=0, missing country=0
OWID CO2: shape=7630x7, uniq iso_code=218, uniq country=218, missing iso_code=0, missing country=0
World Bank indicators: shape=7595x9, uniq iso_code=217, uniq country=217, missing iso_code=0, missing country=0
raw_data merged (co2 + wb_df + undp_df): shape=8389x17, uniq iso_code=240, uniq country=218, missing iso_code=0, missing country=759


To ensure indicator coherence across countries, the merged dataset is restricted to the UNDP country universe. UNDP has the smallest and most consistent ISO coverage (204 countries), while OWID and World Bank datasets include additional ISO codes for which UNDP indicators are unavailable. Allowing these sources to expand the country universe would introduce structurally missing UNDP values and reduce comparability. Therefore, UNDP ISO codes are used as the reference set, and other indicators are merged onto this base.

In [9]:
# Build UNDP country lookup: one row per iso_code with its UNDP country name
undp_204 = (
    undp_df[["iso_code", "country"]]  # take only ID + name
    .dropna(subset=["country"])
    .drop_duplicates()                # one row per (iso_code, country)
)

summarize_ids(undp_204,     "UNDP 204",    iso_col="iso_code", country_col="country")

# After merging co2, wb_df, undp_df we have 3 country cols: 'country', 'country_wb', 'country_undp', etc.
# We don't want multiple versions – we'll rebuild a clean one from 204 undp countries. First we drop all countries
country_cols = [c for c in raw_data.columns if c.startswith("country")]
raw_data = raw_data.drop(columns=country_cols)


# Making a raw dataset with only 204 chosen countries (filter to only keep 204 iso and attaching country names column to it)
raw_data_204 = raw_data.merge(
    undp_204,
    on="iso_code",                          # Match using key identifier of iso_code
    how="inner"                            # Row identities = intersection of left and right keys → only matching iso_codes create rows → both sides fill columns → non-matching iso_codes are dropped
)

summarize_ids(raw_data_204, "raw_data_undp", iso_col="iso_code", country_col="country")

UNDP 204: shape=204x2, uniq iso_code=204, uniq country=204, missing iso_code=0, missing country=0
raw_data_undp: shape=7129x15, uniq iso_code=204, uniq country=204, missing iso_code=0, missing country=0


In building the final analysis dataset, we first cleaned the OWID CO₂ data by removing non-country entries (regional and aggregate rows such as “Africa” or “World”) and any OWID-specific aggregate codes, keeping only country-level records with valid ISO codes. We then combined three sources – OWID CO₂ (environmental pressure), World Bank indicators (socioeconomic and demographic variables), and UNDP human development data (MYS and GNI) – using outer joins on iso_code and year, creating a large merged table (raw_data) with 8,389 rows and 240 distinct ISO codes. The outer joins did not discard information: they kept the union of all entities tracked by these providers. However, both OWID and the World Bank report data not only for UNDP countries, but also for additional “economies” such as small territories, special administrative regions, and other non-UNDP entities. These show up as extra ISO codes in raw_data even after removing explicit World Bank aggregates, because they are still treated as separate economies in the indicator API (e.g. territories with their own codes). To focus on a consistent and well-covered set of countries, we defined our country universe as the 204 economies in the merged UNDP dataset (undp_df), where human development indicators are available. We built a clean lookup of these 204 iso_code–country pairs and then dropped all conflicting country* columns from raw_data, finally applying an inner join on iso_code to keep only those 204 UNDP countries and attach a single, consistent UNDP country name. The resulting table raw_data_undp has 7,129 rows, 204 unique countries, and no missing iso_code or country, preserving all available indicators for those countries while intentionally excluding regional aggregates and non-UNDP economies that would introduce noise and patchy coverage.

In [ ]:
#Saving to avoid running APIs again and again
raw_data_204.to_csv("data/rawdata.csv", index=False)